In [ ]:
# Sumani
# 8-8-2024

# Building and Training a CNN model on MNIST Dataset Using PyTorch

In this notebook, we will walk through the process of building, training, and evaluating a Convolutional Neural Network (CNN) for classifying handwritten digits from the MNIST dataset using PyTorch. We'll cover data loading and preprocessing, model definition, training, evaluation, and visualization of the results.

## Introduction

The MNIST dataset contains 70,000 grayscale images of handwritten digits (0-9), each of size 28x28 pixels. The dataset is divided into 60,000 training images and 10,000 test images. The goal is to classify each image into one of the 10 digit classes.


## Import Libraries

In [ ]:
# Import the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

# Step 1: Data Loading and Preprocessing

We'll start by defining the transformations to apply to the images and loading the MNIST dataset.

#### MNIST Dataset

The MNIST (Modified National Institute of Standards and Technology) dataset is a large database of handwritten digits that is commonly used for training various image processing systems. The dataset contains 70,000 images of handwritten digits (0-9) with 60,000 images in the training set and 10,000 images in the test set. Each image is a 28x28 grayscale image, meaning each pixel value ranges from 0 (black) to 255 (white). The dataset is well-suited for training and testing machine learning models in image recognition tasks, and it has become a standard benchmark for evaluating algorithms.



#### torch.utils.data.DataLoader

torch.utils.data.DataLoader is a utility in PyTorch that provides an efficient way to load and preprocess data. It allows for easy batching, shuffling, and parallel data loading using multiple workers. Key features include:

1. `Batching`: Automatically groups data into batches.
2. `Shuffling`: Randomizes the order of data samples, which is important for training neural networks.
3. `Parallel Loading`: Utilizes multiple CPU cores to load data in parallel, reducing the time spent on data loading.

DataLoader is essential for handling large datasets and ensuring efficient data feeding during the training and evaluation of models.

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load datasets
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

# Define classes
classes = [str(i) for i in range(10)]

# Step 2: Exploratory Data Analysis (EDA)

Let's visualize some sample images from the MNIST dataset.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import torchvision.utils as vutils

real_batch = next(iter(trainloader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training images")
plt.imshow(
    np.transpose(
        vutils.make_grid(real_batch[0][:64], padding=2, normalize=True), (1, 2, 0)
    )
)
plt.show()

# Step 3: Define the Neural Network Model

We will define a simple fully connected neural network with two layers.

In [ ]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = MNISTModel()

In [ ]:
# Create an instance of the model
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MNISTModel()
model = model.to(device)
print(model)

# Step 4: Define the Loss Function and Optimizer

We'll use cross-entropy loss and stochastic gradient descent (SGD) optimizer.

#### CrossEntropyLoss

CrossEntropyLoss is a commonly used loss function for classification problems in neural networks. It combines LogSoftmax and NLLLoss (negative log-likelihood loss) in a single class. CrossEntropyLoss is particularly useful for multi-class classification tasks.

#### Adam

The Adam optimizer is a versatile and powerful choice for training deep learning models. Its adaptive learning rate, moment estimation, and bias correction features make it suitable for a wide range of tasks and datasets, offering efficient and effective optimization.

* Efficient: Adam requires only first-order gradients (derivatives) and has a low memory footprint, making it computationally efficient.



In [ ]:
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 5: Training the Model

Let's define the training loop.

In [ ]:
# Define a function to calculate accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels).item() / len(labels)

running_loss = 0.0
running_acc = 0.0

# Define the training loop
def train(model, device, train_loader, criterion, optimizer, epoch):
    global running_loss
    global running_acc
    running_acc = 0.0
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_acc += accuracy(outputs, labels)
        if (i + 1) % 200 == 0:
            print(f'Epoch {epoch}, Batch {i + 1}, Loss: {running_loss / 200:.4f}, Accuracy: {running_acc / 200:.4f}')
            running_loss = 0.0
            running_acc = 0.0

# Track training loss and accuracy
train_losses = []
train_accuracies = []

# Train the model
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, criterion, optimizer, epoch)
    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(running_acc / len(train_loader))
